In [9]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.weightstats import ttest_ind
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
import statsmodels.api as sm

In [10]:
dpw_hourly = pd.read_csv("data/dpw_hourly_o3.csv")
pema_hourly = pd.read_csv("data/pema_hourly_o3.csv")
pha_hourly = pd.read_csv("data/pha_hourly_o3.csv")

In [11]:
dpw_hourly = dpw_hourly.rename(columns={"o3": "o3_dpw"})
pema_hourly = pema_hourly.rename(columns={"o3": "o3_pema"})
pha_hourly = pha_hourly.rename(columns={"o3": "o3_pha"})

merged = dpw_hourly.merge(pema_hourly, on="datetime_utc", how="inner") \
                   .merge(pha_hourly, on="datetime_utc", how="inner")


In [12]:
merged["mean_o3"] = merged[["o3_dpw", "o3_pema", "o3_pha"]].mean(axis=1)
merged["std_o3"] = merged[["o3_dpw", "o3_pema", "o3_pha"]].std(axis=1)

merged["rsd_o3"] = merged["std_o3"] / merged["mean_o3"].replace(0, pd.NA)
merged.head()

,datetime_utc,o3_dpw,o3_pema,o3_pha,mean_o3,std_o3,rsd_o3
0,2024-12-17 00:00:00+00:00,39.337867,35.763817,16.174917,30.425533,12.470105,0.409857
1,2024-12-17 01:00:00+00:00,40.264133,34.327867,19.326967,31.306322,10.790669,0.344680
2,2024-12-17 02:00:00+00:00,40.435317,38.493650,22.212433,33.713800,10.007677,0.296842
3,2024-12-17 03:00:00+00:00,38.338317,38.445650,22.706650,33.163539,9.056090,0.273074
4,2024-12-17 04:00:00+00:00,42.874350,41.017483,27.082383,36.991406,8.631543,0.233339


In [13]:
cutoff = merged["rsd_o3"].quantile(0.10)
lowest_rsd_hours = merged[merged["rsd_o3"] <= cutoff].copy()

lowest_rsd_hours.to_csv("data/lowest_rsd_hours.csv")